# Self Discover

An implementation of the [Self-Discover paper](https://arxiv.org/pdf/2402.03620.pdf).

Based on [this implementation from @catid](https://github.com/catid/self-discover/tree/main?tab=readme-ov-file)

In [37]:
# from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

In [38]:
# model = ChatOpenAI(temperature=0, model="gpt-4-turbo-preview")
model = ChatOllama(model="qwen:32b-chat-v1.5-q5_K_M")

In [39]:
from langchain import hub
from langchain_core.prompts import PromptTemplate

In [40]:
select_prompt = hub.pull("superfhwl/self-discovery-select")

In [41]:
select_prompt.pretty_print()

选择几个关键的推理模块来解决给定的任务:

所有推理模块说明:
{reasoning_modules}

任务:{task_description}

选择几个对解决上述任务至关重要的模块:


In [42]:
adapt_prompt = hub.pull("superfhwl/self-discovery-adapt")

In [43]:
adapt_prompt.pretty_print()

重新表述和指定每个推理模块，以便它更好地帮助解决任务:

选定模块说明:
{selected_modules}

任务:{task_description}

调整每个推理模块的描述以更好地解决任务:


In [44]:
structured_prompt = hub.pull("superfhwl/self-discovery-structure")

In [45]:
structured_prompt.pretty_print()

将推理模块操作成JSON格式的分步推理计划:

这里有一个例子:

示例任务:

如果你按照这些指示去做，你会回到起点吗?永远面向前方。后退一步。向左走9步。后退两步。向前走6步。向前走4步。后退4步。向右走三步。

示例推理结构:

{
"指令1后的位置":
"指令2后的位置":
"指令n后的位置":
"最终位置是否与起始位置相同?"
}

改编模块描述:
{adapted_modules}

任务:{task_description}

实现一个推理结构，让求解者逐步遵循并得出正确答案。

注意:不要在这个过程中得出结论。你的工作是生成一个计划，以便将来你可以填写它，并得出正确的结论


In [46]:
reasoning_prompt = hub.pull("superfhwl/self-discovery-reasoning")

In [47]:
reasoning_prompt.pretty_print()

按照JSON格式的分步推理计划正确解决任务。通过对给定任务的具体推理，填写键后面的值。不要简单地修改关键字。

逻辑结构:
{reasoning_structure}

任务:{task_description}


In [48]:
reasoning_prompt

PromptTemplate(input_variables=['reasoning_structure', 'task_description'], metadata={'lc_hub_owner': 'superfhwl', 'lc_hub_repo': 'self-discovery-reasoning', 'lc_hub_commit_hash': 'aeb99fab1804a7d5accd3ad464a61a51b0ded25734fd15a071a65f1dc866170f'}, template='按照JSON格式的分步推理计划正确解决任务。通过对给定任务的具体推理，填写键后面的值。不要简单地修改关键字。\n\n逻辑结构:\n{reasoning_structure}\n\n任务:{task_description}')

In [49]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [50]:
select_chain = select_prompt | model | StrOutputParser()

In [51]:
adapt_chain = adapt_prompt | model | StrOutputParser()

In [52]:
structure_chain = structured_prompt | model | StrOutputParser()

In [53]:
reasoning_chain = reasoning_prompt | model | StrOutputParser()

In [54]:
overall_chain = (
    RunnablePassthrough.assign(selected_modules=select_chain)
    .assign(adapted_modules=adapt_chain)
    .assign(reasoning_structure=structure_chain)
    .assign(answer=reasoning_chain)
)

In [55]:
reasoning_modules = [
    "1. 我怎样才能设计一个实验来解决这个问题呢?"
    "2. 把解决这个问题的想法列一个清单，然后一个接一个地应用到这个问题上，看看是否能取得进展. "
    "3. 我怎样才能衡量这个问题的进展呢?"
    "4. 我怎样才能简化这个问题，使它更容易解决?"
    "5. 这个问题背后的关键假设是什么?"
    "6. 每种解决方案的潜在风险和缺点是什么?"
    "7. 对这个问题有什么不同的观点或观点?"
    "8. 这个问题及其解决方案的长期影响是什么?"
    "9. 我怎样才能把这个问题分解成更小、更容易处理的部分呢?"
    "10. 批判性思维:这种风格包括从不同的角度分析问题，质疑假设，评估现有的证据或信息. 它侧重于逻辑推理、基于证据的决策，以及识别思维中的潜在偏见或缺陷. "
    "11. 尝试创造性思维，产生创新的和打破常规的想法来解决问题. 探索非传统的解决方案，超越传统的界限思考，鼓励想象力和独创性. "
    "12. 寻求他人的意见和合作来解决问题. 强调团队合作，开放沟通，利用团队的不同观点和专业知识来提出有效的解决方案. "
    "13. 使用系统思维:将问题视为更大系统的一部分，并理解各种元素的相互联系. 重点是确定影响问题的潜在原因、反馈回路和相互依赖关系，并制定整体解决方案，以解决整个系统. "
    "14. 使用风险分析:评估与问题的不同解决方案或方法相关的潜在风险、不确定性和权衡. 强调评估潜在的后果和成功或失败的可能性，并在平衡分析风险和收益的基础上做出明智的决定. "
    "15. 运用反思性思维:从问题中退后一步，花时间反省和自我反省. 检查可能影响解决问题的个人偏见、假设和心理模型，并从过去的经验中学习，以改进未来的方法. "
    "16. 需要解决的核心问题是什么?"
    "17. 造成这个问题的根本原因或因素是什么?"
    "18. 有没有什么潜在的解决方案或策略是以前尝试过的?如果是，结果和教训是什么?"
    "19. 在解决这个问题的过程中可能会遇到哪些潜在的障碍或挑战?"
    "20. 是否有任何相关的数据或信息，可以提供洞察问题?如果是，有哪些可用的数据来源，如何分析这些数据?"
    "21. 是否有任何利益相关者或个人直接受到这个问题的影响?他们的观点和需求是什么?"
    "22. 需要什么资源(财政、人力、技术等)才能有效地解决这个问题?"
    "23. 如何衡量或评估解决问题的进展或成功?"
    "24. 可以使用哪些指标或度量标准?"
    "25. 问题是技术性的还是实践性的，需要特定的专业知识或技能?或者它更像是一个概念或理论问题?"
    "26. 问题是否涉及物理限制，例如有限的资源、基础设施或空间?"
    "27. 这个问题是否与人类行为有关，比如社会、文化或心理问题?"
    "28. 问题是否涉及决策或计划，需要在不确定或目标相互竞争的情况下做出选择?"
    "29. 这个问题是需要数据分析、建模或优化技术的分析性问题吗?"
    "30. 这个问题是一个需要创造性解决方案和创新的设计挑战吗?"
    "31. 这个问题是否需要解决系统或结构性问题，而不仅仅是个别问题?"
    "32. 这个问题是时间敏感还是紧急，需要立即注意和采取行动?"
    "33. 对于这类问题说明，通常会产生什么样的解决方案?"
    "34. 根据问题说明和当前的最佳解决方案，猜测其他可能的解决方案. "
    "35. 让我们假设当前的最佳解决方案是完全错误的，还有什么其他方式来思考问题规范?"
    "36. 根据您对这类问题说明的了解，修改当前最佳解决方案的最佳方法是什么?"
    "37. 忽略当前的最佳解决方案，创造一个全新的解决方案. "
    "38. 让我们一步一步来思考. "
    "39. 让我们制定一个循序渐进的计划，并用好的符号和解释来实施它. "
]

reasoning_modules_str = "\n".join(reasoning_modules)

In [60]:

# task_example = "丽莎有10个苹果。她给了她的朋友3个苹果，然后又从商店买了5个苹果。丽莎现在有几个苹果?"

# task_example = """这个SVG路径元素<path d="M 55.57,80.69 L 57.38,65.80 M 57.38,65.80 L 48.90,57.46 M 48.90,57.46 L
# 45.58,47.78 M 45.58,47.78 L 53.25,36.07 L 66.29,48.90 L 78.69,61.09 L 55.57,80.69"/>绘制了一个：
# (A)圆(B)七边形(C)六边形(D)风筝(E)线(F)八边形(G)五边形(H)矩形(I)扇形(J)三角形"""

# task_example = "不借助3D工具，直接输出一个Pixar USD 格式的文件文本，该文件中描述了一个黄色的box形状，该box在场景的正中央。长宽高分别为 长1米，宽1米，高1米。"

# task_example = """一只母鸡每天可以生一只鸡蛋。鸡蛋可以被拿去贩卖，也可以拿来孵化更多的鸡。但是鸡蛋不能存储，必须当天销售或者当天开始孵化
# 在孵化的过程中，负责孵化鸡蛋的这只母鸡无法产蛋。
# 假设每个鸡蛋在孵化10天后即可孵化出一只小鸡，小鸡成长成母鸡需要30天。
# 第一天有10只母鸡，请规划一个策略，在第1000天时，贩卖出尽量多的鸡蛋。
# 不考虑生病和死亡的情况，假设鸡舍空间足够大可以容纳所有的鸡，假设有足够的饲料喂养所有的鸡
# 生成一段python代码，实现上述策略，并计算出结果"""

# task_example = """给你一个整数数组 coins ，表示不同面额的硬币；以及一个整数 amount ，表示总金额。
# 计算并返回可以凑成总金额所需的 最少的硬币个数 。如果没有任何一种硬币组合能组成总金额，返回 -1 。
# 你可以认为每种硬币的数量是无限的。
# 请输出一段python代码，实现上述算法"""

task_example = """这里中国，现在是夏天。
有一间卧室，门在东北角，窗在正南。卧室南北长3.5米，东西长2.8米。
卧室里有一张床，长度是2.1米，宽度是1.5米。有一个书桌，长度是1米，宽度是0.6米，高度是0.8米。有一张靠背椅，坐高0.45米。有一个衣柜，长1.2米，宽0.4米，高2米，衣柜是双开门。
请设计一个布局摆放上述家具，满足如下条件：1. 床可以从三面上下。2. 坐在靠背椅上使用书桌写字，比较便利，并且光线充足。3. 衣柜可以正常使用，不会影响到衣柜开门。
请生成一段json文件，通过上述家具的坐标，描述上述布局的俯视图"""


output = overall_chain.invoke(
    {"task_description": task_example, "reasoning_modules": reasoning_modules_str}
)

for k,v in output.items():
    print(k)
    print(v)

task_description
这里中国，现在是夏天。
有一间卧室，门在东北角，窗在正南。卧室南北长3.5米，东西长2.8米。
卧室里有一张床，长度是2.1米，宽度是1.5米。有一个书桌，长度是1米，宽度是0.6米，高度是0.8米。有一张靠背椅，坐高0.45米。有一个衣柜，长1.2米，宽0.4米，高2米，衣柜是双开门。
请设计一个布局摆放上述家具，满足如下条件：1. 床可以从三面上下。2. 坐在靠背椅上使用书桌写字，比较便利，并且光线充足。3. 衣柜可以正常使用，不会影响到衣柜开门。
请生成一段json文件，通过上述家具的坐标，描述上述布局的俯视图
reasoning_modules
1. 我怎样才能设计一个实验来解决这个问题呢?2. 把解决这个问题的想法列一个清单，然后一个接一个地应用到这个问题上，看看是否能取得进展. 3. 我怎样才能衡量这个问题的进展呢?4. 我怎样才能简化这个问题，使它更容易解决?5. 这个问题背后的关键假设是什么?6. 每种解决方案的潜在风险和缺点是什么?7. 对这个问题有什么不同的观点或观点?8. 这个问题及其解决方案的长期影响是什么?9. 我怎样才能把这个问题分解成更小、更容易处理的部分呢?10. 批判性思维:这种风格包括从不同的角度分析问题，质疑假设，评估现有的证据或信息. 它侧重于逻辑推理、基于证据的决策，以及识别思维中的潜在偏见或缺陷. 11. 尝试创造性思维，产生创新的和打破常规的想法来解决问题. 探索非传统的解决方案，超越传统的界限思考，鼓励想象力和独创性. 12. 寻求他人的意见和合作来解决问题. 强调团队合作，开放沟通，利用团队的不同观点和专业知识来提出有效的解决方案. 13. 使用系统思维:将问题视为更大系统的一部分，并理解各种元素的相互联系. 重点是确定影响问题的潜在原因、反馈回路和相互依赖关系，并制定整体解决方案，以解决整个系统. 14. 使用风险分析:评估与问题的不同解决方案或方法相关的潜在风险、不确定性和权衡. 强调评估潜在的后果和成功或失败的可能性，并在平衡分析风险和收益的基础上做出明智的决定. 15. 运用反思性思维:从问题中退后一步，花时间反省和自我反省. 检查可能影响解决问题的个人偏见、假设和心理模型，并从过去的经验中学习，以改进未来的方法. 16. 需要解决的核心问题是什么?17. 造成这个问题的根本

In [57]:
import pxr.Usd as usd

# 创建新的 USD 根层
stage = usd.Stage.CreateInMemory("RoomLayout.usda")
root_layer = stage.GetRootLayer()

# 添加房间的基本几何体（长方体）
room_prim = stage.DefinePrim("/Room", "Box")
room_geom = room_prim.CreateAttribute("geometry", usd.Attribute.Tokens)
room_geom.Set(["box"])
room_prim.GetAttribute("extent").Set([3.5, 2.8, 3])  # 长度、宽度和高度

# 添加家具，例如床
bed_prim = stage.DefinePrim("/Bed", "Box")
bed_geom = bed_prim.CreateAttribute("geometry", usd.Attribute.Tokens)
bed_geom.Set(["box"])
bed_prim.GetAttribute("extent").Set([2.1, 1.5, 0.8])  # 长度、宽度和高度
bed_prim.AddTransformOp().SetTranslation([-1.7, 0.9, 0])  # 床放在靠窗的一侧

# 添加书桌
desk_prim = stage.DefinePrim("/Desk", "Box")
desk_geom = desk_prim.CreateAttribute("geometry", usd.Attribute.Tokens)
desk_geom.Set(["box"])
desk_prim.GetAttribute("extent").Set([1, 0.6, 0.8])  # 长度、宽度和高度
desk_prim.AddTransformOp().SetTranslation([-1.7, -1.25, 0])  # 在西南角，面向窗户

# 添加靠背椅
chair_prim = stage.DefinePrim("/Chair", "Box")
chair_geom = chair_prim.CreateAttribute("geometry", usd.Attribute.Tokens)
chair_geom.Set(["box"])
chair_prim.GetAttribute("extent").Set([0.5, 0.5, 0.45])  # 假设椅子尺寸
chair_prim.AddTransformOp().SetTranslation([-1.2, -1.25, 0])  # 在书桌旁边

# 添加衣柜
wardrobe_prim = stage.DefinePrim("/Wardrobe", "Box")
wardrobe_geom = wardrobe_prim.CreateAttribute("geometry", usd.Attribute.Tokens)
wardrobe_geom.Set(["box"])
wardrobe_prim.GetAttribute("extent").Set([1.2, 0.4, 2])  # 长度、宽度和高度
wardrobe_prim.AddTransformOp().SetTranslation([-0.25, -0.9, 0])  # 在房间另一侧

# 提交更改并保存 USD 文件
stage.GetRootLayer().Save()

AttributeError: type object 'Attribute' has no attribute 'Tokens'